In [69]:
deployment = "gpt4"
model = "gpt-4"

In [2]:
!pip install openai==1.3.6

In [70]:
def get_current_cluster_state(cluster_name):
    print(f"cluster:{cluster_name}")
    return  """ERROR: Failed to pull image "chaocai/docker/dsp:latest"""

In [71]:
import os
from openai import AzureOpenAI
    
client = AzureOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),  
    api_version=os.getenv("OPENAI_API_VERSION"),
    azure_endpoint = os.getenv("OPENAI_API_BASE")
    )

In [72]:
import openai
import json


funcs = {"get_current_cluster_state": get_current_cluster_state}


def run(input):
    msg=[{"role":"user","content":input}]
    ret = run_conversation(msg)
    return ret.content

def run_conversation(msg):
    response = client.chat.completions.create(
        model=deployment,
        messages=msg,
        functions=[
            {
                    "name": "get_current_cluster_state",
                    "description": "Get the current state in a given cluster",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "cluster_name": {
                                "type": "string",
                                "description": "the name of the cluster",
                            },

                        },
                        "required": ["cluster_name"],
                    }
              
            }
        ],
        function_call="auto",
    )

    response_message = response.choices[0].message
    
    function_call = response_message.function_call
  
    # 如果不需要调用function，则直接返回结果
    if not function_call:
        return response_message
    function_name = function_call.name
    function_to_call = funcs[function_name] 
    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)                
    msg.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    msg.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    ) 
   
    return run_conversation(msg)
   

In [73]:
#run("Hello")
print(run("What's wrong with the cluster 'DSP'? And if there's an error, give me some suggestion. "))

cluster:DSP
The issue with the cluster 'DSP' appears to be related to an image pulling problem. Specifically, it failed to pull the image `"chaocai/docker/dsp:latest"`.

Here are a few suggestions to troubleshoot and potentially resolve the issue:

1. **Check the Image Repository**: Ensure that the image repository `chaocai/docker/dsp` exists and that the `latest` tag is available within the repository.

2. **Access Permissions**: Verify that you have the correct access permissions to the image repository. If it's a private repository, you'll need to ensure that the cluster has the necessary credentials to access it.

3. **Network Connectivity**: Confirm that the network configuration is allowing the nodes in the DSP cluster to reach the image repository. This includes checking for any firewall rules, security groups, or other network policies that could prevent access.

4. **Image Pull Policy**: Check the image pull policy for the pod to determine when the image should be pulled. If i